# MCP Server example: One Agent Multiple Servers.

this notebook is an example where you can run multiple MCP server via an agent toolcall.

In [ ]:
# import necessary agent and client libraries
import os
from contextlib import asynccontextmanager
from datetime import timedelta
from pathlib import Path
from typing import Any, AsyncIterator, Literal

import dotenv
import nest_asyncio
from mcp.client.session import ClientSession
from mcp.client.sse import sse_client
from mcp.client.stdio import StdioServerParameters, stdio_client

from autogen import LLMConfig
from autogen.agentchat import AssistantAgent, ConversableAgent
from autogen.agentchat.group import (
    AgentTarget,
    ReplyResult,
)
from autogen.mcp import create_toolkit

nest_asyncio.apply()

define LLMConfig for the agent. make sure to set the OPENAI_API_KEY in your environment variables. you can use a .env file or set it directly in your environment

In [ ]:
dotenv.load_dotenv()
llm_config = LLMConfig(model="o4-mini", api_type="openai", api_key=os.getenv("OPENAI_API_KEY"))

define server file paths for stdio client and server url for sse client.

In [ ]:
FILESYSTEM_SERVER_PATH = Path("mcp/mcp_filesystem.py")
WIKIPEDIA_SERVER_URL = "http://localhost:8000/sse"

In [ ]:
EncodingErrorHandlerType = Literal["strict", "ignore", "replace"]

DEFAULT_TEXT_ENCODING = "utf-8"
DEFAULT_TEXT_ENCODING_ERROR_HANDLER: EncodingErrorHandlerType = "strict"

DEFAULT_HTTP_REQUEST_TIMEOUT = 5
DEFAULT_SSE_EVENT_READ_TIMEOUT = 60 * 5

DEFAULT_STREAMABLE_HTTP_REQUEST_TIMEOUT = timedelta(seconds=30)
DEFAULT_STREAMABLE_HTTP_SSE_EVENT_READ_TIMEOUT = timedelta(seconds=60 * 5)

define connection params

In [ ]:
# add MCP server params
mcp_connections: dict = {
    "FilesystemServer": {
        "command": "python3",
        "args": [FILESYSTEM_SERVER_PATH],
        "transport": "stdio",
    },
    "WikipediaServer": {
        "url": WIKIPEDIA_SERVER_URL,
        "transport": "sse",
    },
}

add methods to create client sessions

In [ ]:
@asynccontextmanager
async def create_stdio_mcp_session(
    *,
    command: str,
    arguments: list[str],
    environment: dict[str, str] | None = None,
    working_dir: str | Path | None = None,
    encoding: str = DEFAULT_TEXT_ENCODING,
    encoding_error_handler: EncodingErrorHandlerType = DEFAULT_TEXT_ENCODING_ERROR_HANDLER,
    session_options: dict[str, Any] | None = None,
) -> AsyncIterator[ClientSession]:
    """
    Create a new session to an MCP server using stdio

    Args:
        command: Command to execute
        arguments: Arguments for the command
        environment: Environment variables for the command
        working_dir: Working directory for the command
        encoding: Character encoding
        encoding_error_handler: How to handle encoding errors
        session_options: Additional keyword arguments to pass to the ClientSession
    """

    environment = environment or {}
    if "PATH" not in environment:
        environment["PATH"] = os.environ.get("PATH", "")

    server_params = StdioServerParameters(
        command=command,
        args=arguments,
        env=environment,
        cwd=working_dir,
        encoding=encoding,
        encoding_error_handler=encoding_error_handler,
    )

    async with (
        stdio_client(server_params) as (reader, writer),
        ClientSession(reader, writer, **(session_options or {})) as session,
    ):
        yield session


@asynccontextmanager
async def create_sse_mcp_session(
    *,
    url: str,
    headers: dict[str, Any] | None = None,
    timeout: float = DEFAULT_HTTP_REQUEST_TIMEOUT,
    sse_read_timeout: float = DEFAULT_SSE_EVENT_READ_TIMEOUT,
    session_options: dict[str, Any] | None = None,
    httpx_client_factory: Any = None,
) -> AsyncIterator[ClientSession]:
    """
    Create a new session to an MCP server using SSE

    Args:
        url: URL of the SSE server
        headers: HTTP headers to send to the SSE endpoint
        timeout: HTTP timeout
        sse_read_timeout: SSE read timeout
        session_options: Additional keyword arguments to pass to the ClientSession
        httpx_client_factory: Custom factory for httpx.AsyncClient (optional)
    """
    kwargs = {}
    if httpx_client_factory is not None:
        kwargs["httpx_client_factory"] = httpx_client_factory

    async with (
        sse_client(url, headers, timeout, sse_read_timeout, **kwargs) as (reader, writer),
        ClientSession(reader, writer, **(session_options or {})) as session,
    ):
        yield session

open a mcp session

In [ ]:
@asynccontextmanager
async def open_mcp_session(
    config,
) -> AsyncIterator[ClientSession]:
    """
    Open a new session to an MCP server.

    Args:
        config: Configuration dictionary for the connection

    Raises:
        KeyError: If required parameters for the specified transport are missing
        NotImplementedError: If transport is not recognized

    Yields:
        A ClientSession
    """
    transport_type = config["transport"]
    if transport_type == "sse":
        if "url" not in config:
            raise KeyError("'url' parameter is required for SSE connection")
        async with create_sse_mcp_session(
            url=config["url"],
            headers=config.get("headers"),
            timeout=config.get("timeout", DEFAULT_HTTP_REQUEST_TIMEOUT),
            sse_read_timeout=config.get("sse_read_timeout", DEFAULT_SSE_EVENT_READ_TIMEOUT),
            session_options=config.get("session_options"),
            httpx_client_factory=config.get("httpx_client_factory"),
        ) as session:
            yield session
    elif transport_type == "stdio":
        if "command" not in config:
            raise KeyError("'command' parameter is required for stdio connection")
        if "arguments" not in config:
            raise KeyError("'arguments' parameter is required for stdio connection")
        async with create_stdio_mcp_session(
            command=config["command"],
            arguments=config["arguments"],
            environment=config.get("environment"),
            working_dir=config.get("working_dir"),
            encoding=config.get("encoding", DEFAULT_TEXT_ENCODING),
            encoding_error_handler=config.get("encoding_error_handler", DEFAULT_TEXT_ENCODING_ERROR_HANDLER),
            session_options=config.get("session_options"),
        ) as session:
            yield session
    else:
        raise NotImplementedError(f"Unsupported transport: {transport_type}. Must be one of: 'stdio', 'sse'")

define 'mcp_agent' and assistant agent for tool execution.

In [ ]:
system_message = """
You are a mcp agent that can use mcp tools to analyze the content of the file and return the result.
you have two mcp servers connected to you.
1. FilesystemServer
2. WikipediaServer

you can use the tools of both the servers to analyze the content of the file and return the result.

# Note:
    - Identify the server name by analyzing the human question/initial message.
"""

mcp_agent = ConversableAgent(
    name="mcp_agent",
    system_message=system_message,
    llm_config=llm_config,
)

assistant = ConversableAgent(
    name="assistant",
    system_message="""You are an assistant agent that can use two mcp servers connected to you.
    1. FilesystemServer
    2. WikipediaServer
    """,
    llm_config=llm_config,
    max_consecutive_auto_reply=3,
)

In [ ]:
tool_prompt = """
    Run the MCP agent for the given server name and query.
    This function executes the MCP agent with the specified query and server name,
    returning the result of the MCP agent's processing.
    Args:
        query (str): The query to be processed by the MCP agent.
        server_name (str): The name of the server that is connected to the MCP agent.
    Returns:
        ReplyResult: The result of the MCP agent's processing.
"""


@mcp_agent.register_for_llm(description=tool_prompt)
@assistant.register_for_execution(description=tool_prompt)
async def run_mcp_agent_to_client(query: str, server_name: str) -> ReplyResult:
    if server_name not in mcp_connections:
        raise KeyError(
            f"Couldn't find a server with name '{server_name}', expected one of '{list(mcp_connections.keys())}'"
        )
    async with open_mcp_session(mcp_connections[server_name]) as session:
        await session.initialize()
        toolkit = await create_toolkit(session=session)
        agent = AssistantAgent(
            name="assistant",
            llm_config=llm_config,
            human_input_mode="NEVER",
        )
        toolkit.register_for_llm(agent)
        # Make a request using the MCP tool
        result = await agent.a_run(
            message=query,
            tools=toolkit.tools,
            max_turns=2,
        )
        res = await result.process()
        return ReplyResult(
            message=str(res),
            target=AgentTarget(mcp_agent),
        )

In [ ]:
res = mcp_agent.initiate_chat(
    message="search wikipedia for the term 'demon slayer movies'", recipient=assistant, max_turns=3
)